<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-15.-Context-Managers-and-else-Blocks" data-toc-modified-id="Chapter-15.-Context-Managers-and-else-Blocks-1">Chapter 15. Context Managers and else Blocks</a></span><ul class="toc-item"><li><span><a href="#else-Blocks" data-toc-modified-id="else-Blocks-1.1">else Blocks</a></span><ul class="toc-item"><li><span><a href="#for-loop" data-toc-modified-id="for-loop-1.1.1">for loop</a></span></li><li><span><a href="#try/except-Blocks" data-toc-modified-id="try/except-Blocks-1.1.2">try/except Blocks</a></span></li></ul></li><li><span><a href="#with-Blocks" data-toc-modified-id="with-Blocks-1.2">with Blocks</a></span></li><li><span><a href="#Context-Managers" data-toc-modified-id="Context-Managers-1.3">Context Managers</a></span></li><li><span><a href="#Context-manager-as-a-decorator:-@contextmanager" data-toc-modified-id="Context-manager-as-a-decorator:-@contextmanager-1.4">Context manager as a decorator: @contextmanager</a></span></li></ul></li><li><span><a href="#Chapter-16.-Coroutines" data-toc-modified-id="Chapter-16.-Coroutines-2">Chapter 16. Coroutines</a></span><ul class="toc-item"><li><span><a href="#simple-coroutine" data-toc-modified-id="simple-coroutine-2.1">simple coroutine</a></span></li><li><span><a href="#Coroutine-states" data-toc-modified-id="Coroutine-states-2.2">Coroutine states</a></span></li><li><span><a href="#a-running-average-coroutine" data-toc-modified-id="a-running-average-coroutine-2.3">a running average coroutine</a></span><ul class="toc-item"><li><span><a href="#Using-@wraps-decorator" data-toc-modified-id="Using-@wraps-decorator-2.3.1">Using @wraps decorator</a></span></li></ul></li><li><span><a href="#Coroutine-Termination-and-Exception-Handling" data-toc-modified-id="Coroutine-Termination-and-Exception-Handling-2.4">Coroutine Termination and Exception Handling</a></span><ul class="toc-item"><li><span><a href="#generator.throw()" data-toc-modified-id="generator.throw()-2.4.1">generator.throw()</a></span></li><li><span><a href="#generator.close()" data-toc-modified-id="generator.close()-2.4.2">generator.close()</a></span></li></ul></li><li><span><a href="#returning-values-from-coroutines" data-toc-modified-id="returning-values-from-coroutines-2.5">returning values from coroutines</a></span><ul class="toc-item"><li><span><a href="#returning-values-from-coroutines-without-using-yield-from" data-toc-modified-id="returning-values-from-coroutines-without-using-yield-from-2.5.1">returning values from coroutines without using <code>yield from</code></a></span></li><li><span><a href="#usingyield-from" data-toc-modified-id="usingyield-from-2.5.2">using<code>yield from</code></a></span></li></ul></li><li><span><a href="#Taxi-simulator" data-toc-modified-id="Taxi-simulator-2.6">Taxi simulator</a></span></li></ul></li></ul></div>

# Chapter 15. Context Managers and else Blocks

## else Blocks

### for loop
The else block will run only if and when the for loop runs to completion (i.e., not if the for is aborted with a break)

In [622]:
from collections import namedtuple
gum=namedtuple('gum',['flavor'])

my_list = [gum('cherry'), gum('vanilla')]
for item in my_list:
    if item.flavor == 'banana':
        break
else:
    raise ValueError('No banana flavor found!')

ValueError: No banana flavor found!

In [621]:
gum('cherry')

gum(flavor='cherry')

### try/except Blocks
The else block will only run if no exception is raised in the try block.

In [623]:
def dangerous_call(): pass
def after_call():print('Phew!')

Instead of this...

In [624]:
try:
    dangerous_call()
    after_call()
except OSError:
    log('OSError...')

Phew!


 For clarity and correctness, the body of a try block should only have the statements that may generate the expected exceptions. This is much better:

In [625]:
try:
    dangerous_call()
except OSError:
    log('OSError...')
else:
    after_call()

Phew!


In [626]:
map(lambda x: x in [1,2])

TypeError: map() must have at least two arguments.

## with Blocks
The with statement was designed to simplify the try/finally pattern, which guarantees that some operation is performed after a block of code, even if the block is aborted because of an exception, a return or sys.exit() call. 

In [628]:
with open('mirror.py') as fp: 
    src = fp.read()
    print(fp.closed)          
print(fp.closed)

False
True


## Context Managers

In [630]:
import sys

class LookingGlass:

    def __enter__(self):
        self.original_write = sys.stdout.write   
        sys.stdout.write = self.reverse_write   
        print('in enter')
        return 'JABBERWOCKY'   

    def reverse_write(self, text):   
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):   
        sys.stdout.write = self.original_write    
        if exc_type is ZeroDivisionError:   
            print('Please DO NOT divide by zero!')
            return True   
        print('in exit')
         

In [637]:
with LookingGlass() as what:    # __enter__ called here
    print('Alice, Kitty and Snowdrop')
    raise ZeroDivisionError   # uncomment to pass ZeroDivisionError as exc_type 
    print(what)                 # __exit__ called here

retne ni
pordwonS dna yttiK ,ecilA
Please DO NOT divide by zero!


In [632]:
print('Back to normal.')

Back to normal.


## Context manager as a decorator: @contextmanager
contextlib.contextmanager decorator wraps the function in a class that implements the `__enter__` and `__exit__` methods

In [642]:
import contextlib
import sys

@contextlib.contextmanager
def looking_glass():
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])
        
    print('before yield')
    sys.stdout.write = reverse_write
    msg = ''
    try:
        print('...gnidleiy')
        yield 'JABBERWOCKY'
    except ZeroDivisionError:   
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write  
        if msg:
            print(msg)   
        print('after yield')

In [643]:
with looking_glass() as what:    # before yield returns
    print('Alice, Kitty and Snowdrop')
    raise ZeroDivisionError   # uncomment to pass ZeroDivisionError as exc_type 
    print(what)                 # after yield returns
what

before yield
yielding...
pordwonS dna yttiK ,ecilA
Please DO NOT divide by zero!
after yield


'JABBERWOCKY'